In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.3 MB/s eta 0:00:00


In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

In [3]:
train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')
test = pd.read_csv('test.csv')

In [4]:
train = train[['text','stars']]

In [ ]:
train.drop_duplicates(subset='text',inplace=True)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18000 non-null  object
 1   stars   18000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 281.4+ KB


In [6]:
possible_labels = train.stars.unique()
possible_labels

array([5, 4, 3, 2, 1])

In [7]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{5: 0, 4: 1, 3: 2, 2: 3, 1: 4}

In [8]:
X_train, X_val, y_train, y_val = train_test_split(train.index.values, 
                                                  train.stars.values, 
                                                  test_size=0.10, 
                                                  random_state=42, 
                                                  stratify=train.stars.values)

In [9]:
train['data_type'] = ['not_set']*train.shape[0]

train.loc[X_train, 'data_type'] = 'train'
train.loc[X_val, 'data_type'] = 'val'

<ipython-input-9-36a326364770>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['data_type'] = ['not_set']*train.shape[0]


In [10]:
train.groupby(['stars', 'stars', 'data_type']).count()

text
stars stars data_type      
1     1     train      2393
            val         266
2     2     train      1284
            val         142
3     3     train      1772
            val         197
4     4     train      3607
            val         401
5     5     train      7144
            val         794

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    train[train.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    train[train.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train[train.data_type=='train'].stars.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(train[train.data_type=='val'].stars.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [13]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [14]:
len(dataset_train), len(dataset_val)

(16200, 1800)

In [15]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=6,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [17]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [19]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [20]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [22]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [23]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1013 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.8637171213506829
Validation loss: 0.7545768254626114
F1 Score (Weighted): 0.6795930920000809


Epoch 2:   0%|          | 0/1013 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.654190214312206
Validation loss: 0.7300146029586285
F1 Score (Weighted): 0.699584630099294


Epoch 3:   0%|          | 0/1013 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.5616043685895905
Validation loss: 0.7652329509764646
F1 Score (Weighted): 0.6933984084558836


In [24]:
encoded_data_val2 = tokenizer.batch_encode_plus(
    valid.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


generate predictions for the validation file provided

In [25]:
input_ids_val2 = encoded_data_val2['input_ids']
attention_masks_val2 = encoded_data_val2['attention_mask']
labels_val2 = torch.tensor(valid.stars.values)

dataset_val2 = TensorDataset(input_ids_val2, attention_masks_val2, labels_val2)

In [26]:
len(dataset_val2)

2000

In [27]:
dataloader_validation2 = DataLoader(dataset_val2, 
                                   sampler=SequentialSampler(dataset_val2), 
                                   batch_size=batch_size)

In [28]:
val_loss2, predictions2, true_vals2 = evaluate(dataloader_validation2)
val_f1_2 = f1_score_func(predictions2, true_vals2)

In [29]:
val_f1_2

0.7035322192922143

In [30]:
predictions2

array([[-1.8598237 , -1.6848526 , -1.966887  , -1.5120645 ,  1.3498056 ,
         4.9168434 ],
       [-1.9200015 , -1.7251363 , -2.254232  , -1.413583  ,  1.593894  ,
         5.115092  ],
       [-0.7803915 ,  4.5073457 ,  1.6699312 , -0.5523328 , -2.4819329 ,
        -1.6555026 ],
       ...,
       [-1.6006887 , -2.3193913 , -2.370536  , -1.3516475 ,  2.1552758 ,
         5.1190724 ],
       [-1.5259871 ,  4.223163  ,  1.819562  , -0.2522668 , -2.1677983 ,
        -1.9252928 ],
       [-1.6579591 , -1.4205419 , -1.9392072 , -1.683027  ,  0.93466616,
         5.058235  ]], dtype=float32)

In [31]:
predictions2[900].argmax(0)

5

In [32]:
true_vals2[0]

5

In [33]:
val_label = []
for i in range(len(predictions2)):
  val_label.append(predictions2[i].argmax(0))


In [34]:
len(val_label)

2000

In [35]:
pred_df = pd.DataFrame({'stars': val_label, 'review_id': valid['review_id']})
pred_df.to_csv('pred_validation.csv', index=False)

In [36]:
pred_df

,stars,review_id
0,5,xbpsUgAC6eNmxShdG60IUw
1,5,ev6_ZY6iCx1qGY3bn9fbYQ
2,1,wO8VL5RqQ76hhRXqwHvqLw
3,4,5eeU8UMQENJfP1_sVf0rPA
4,1,HiPOnGlZa6vafsJpP97Ukg
...,...,...
1995,5,129GVqt5lWJBef5mOWl2dQ
1996,3,fgnO2_6d3f0oXk1JDhY1Vw
1997,5,PeCeOrVchdnztNdU93lhdA
1998,1,MNFXDAz2CwBHFSmtVMcIRw


Generate predictions for the test file provide

In [37]:
encoded_data_test = tokenizer.batch_encode_plus(
    test.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [38]:
test['stars'] = 0

In [39]:
test.head()

,business_id,cool,date,funny,review_id,text,useful,user_id,stars
0,V-qDa2kr5qWdhs7PU-l-3Q,0,2013-05-29,0,fBHWLNEJmhk6AkzmfLwWcw,Would like to give this more stars - usually I...,1,1pigoFijaHVWGrQl1_tYjw,0
1,C1zlvNlxlGZB8g0162QslQ,0,2012-03-02 15:51:49,0,ldEQ02aP1OeSa5N2beseNg,My wife and I took some friends here after din...,0,BKWPuPZFcGmgjRFRzoq1pw,0
2,0FOON_PNvG0ZxIZh6Jcv2A,0,2013-09-24 20:31:37,0,0oGr6v9VjtRsRsROGMoWTA,My husband and I had lunch here for the first ...,0,BYVYXKqNs-vv-N1ZhRMs0g,0
3,r49iBfbnfoK7yt4rdsL_7g,0,2018-10-20 01:34:08,0,eg5eJ5HmqXuzkxucnKvMTw,I love coming here with my friends! Great for ...,2,dpzmyNglDMeTgV3T5ylUSQ,0
4,xnLNPkL7bbdhD842T4oPqg,0,2016-09-25,1,BNDAe34Mxj--Brkzcfi4QA,Make sure that you double check how much these...,1,yk9wx31bfMEe_IXB8Q-ylA,0


In [40]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test.stars.values)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [41]:
dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [42]:
fake_loss, predictions_test, fake_vals = evaluate(dataloader_test)



In [43]:
test_label = []
for i in range(len(predictions_test)):
  test_label.append(predictions_test[i].argmax(0))

In [44]:
len(test)

4000

In [45]:
pred_test = pd.DataFrame({'stars': test_label, 'review_id': test['review_id']})
pred_test.to_csv('pred.csv', index=False)

In [46]:
pred_test.head()

,stars,review_id
0,3,fBHWLNEJmhk6AkzmfLwWcw
1,3,ldEQ02aP1OeSa5N2beseNg
2,5,0oGr6v9VjtRsRsROGMoWTA
3,4,eg5eJ5HmqXuzkxucnKvMTw
4,1,BNDAe34Mxj--Brkzcfi4QA
